## LHC collective effects with Xsuite
* In this notebook we will study the TMCI threshold for a single bunch using the LHC impedance model at injection energy
* The TMCI or Transverse Mode Coupling instability is a type of strong headtail instability $\rightarrow$ [See X. Buffat's CAS lecture](https://indico.cern.ch/event/1466612/contributions/6448846/)
  * It is triggerd due to impedance when moving above a certain intensity threshold:
  * This threshold $\Upsilon$ is directly proportional $N$ to intensity and the transverse dipolar wake $W_{\perp}^{dip}$: 
  $$\Upsilon \propto N \cdot W_{\perp}^{dip}$$

* To observe the instability threshold, we will do an **intensity scan $\{N_1,\ ..., \ N_n\}$** varying the single bunch intensities and observing the **tune shift** and **average transverse position** across several turns via Xsuite tracking:
    * The LHC will be modeled as a `one-turn-map`, collapsing all the elements into one position
    * The impedance will be added in it's time-domain form, a `wake function` of units $\text{[v/pC/mm]}$ as a function of time in $\text{[ns]}$, stored in a table.
    * The average turn-by-turn bunch position (horizontal or vertical) and other observables are saved in the so-called `CollectiveMonitor`

* The instability is observed when:
  * a tune shift $\Delta Q$ is observed in the mode-0 comparable to the syncrotron tune $Q_s$ $\rightarrow$ leading to mode coupling
  * an exponential growth is present in the transverse average positions $\overline{x}, \ \overline{y}$

### Import the necessary packages

In [ ]:
import os

import numpy as np
import pandas as pd
import h5py

import pickle

import xtrack as xt
import xpart as xp
import xwakes as xw
import xobjects as xo

from scipy.constants import c as clight

import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib ipympl

### Set the simulation parameters
* LHC machine parameters: optics, acceleration, RF
* Bunch parameters: bunch length, intensities, bins to 
* Numerical parameters: number of slices, macroparticles and turns

In [ ]:
# !start-simulation-settings!

##### Machine parameters
circumference = 27000.0
machine_radius = circumference / (2*np.pi)

##### Parameters for the impedance model
plane = 'x'             
wake_type = 'dipolar'
wake_table = 'data/wake_lhc_injection.dat'

##### Initial offset to the particles x coordinate
initial_offset = 10.0e-6

##### Acceleration parameters
energy_gain_per_turn = 0
main_rf_phase = 180

##### RF parameters
h_RF = np.array([35640,])
V_RF = np.array([8.0e6])
dphi_RF = np.array([main_rf_phase,])
f_rev = 299792458 / circumference
omega_rev = 2*np.pi*f_rev
f_RF = np.array([f_rev*h for h in h_RF])

##### Optics parameters
alphap = 3.48e-4

Qx_frac = 0.275
Qy_frac = 0.295
Qx_int = 64
Qy_int = 59

Qx = Qx_int + Qx_frac
Qy = Qy_int + Qy_frac

##### Chromaticity
chromaticity = 0
print('Qp:', chromaticity)

##### Bunch parameters
p0c = 450.0e9
bucket_length = circumference / h_RF[0]
nemitt_x = 2.0e-6
nemitt_y = 2.0e-6
taub = 1.0e-9    # Full bunch length (4*sigma_z)
sigma_z = taub * clight / 4

# We use a limited amount of MP to have a general view of the wake effects
n_macroparticles = int(10_000)
num_slices = int(100)

# Bunch intensity scan
delta_bint = 0.3e12
bunch_intensity_scan = np.arange(0.1e12, 2.3e12, delta_bint)
print('Bunch intensity scan:', bunch_intensity_scan)

# Number of turns simulated
number_of_turns = 3_000

# Plot or not some plots (wakefield, emittance growth)
flag_plot = True

# Restart tracking simulations even if they were already done
flag_restart_sim = True

# !end-simulation-settings!

### Add the LHC wake model at injection
* Import it as a wake table à la HEADTAIL
* Configure for tracking in Xwakes
* Plot the imported wake

In [ ]:
# Read wake table
wake_hllhc = xw.read_headtail_file(wake_table, ['time', 'dipole_x', 'dipole_y', 'quadrupolar_x', 'quadrupolar_y'])
wake_for_tracking_hllhc = xw.WakeFromTable(table=wake_hllhc, columns=['dipole_x'])


In [ ]:
# Configure the wake for tracking
wake_for_tracking = wake_for_tracking_hllhc
wake_for_tracking.configure_for_tracking(
    zeta_range=(-0.375, 0.375),
    num_slices=num_slices,
    num_turns=1,
)

In [ ]:
# Plot the wake model
if flag_plot:
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.plot(wake_hllhc['time'], np.real(wake_hllhc['dipole_x']), marker=None,c='b', label='LHC wake model')

    ax.set_xlabel('Time [s]')
    ax.set_ylabel('Dipolar wake amplitude [$V C^{-1} m^{-1}$]')

    ax.axvline(1.e-9, ymax=0.7, c='r', ls='--')
    ax.axvline(25.e-9,  ymax=0.7, c='darkorange', ls='--')
    ax.axvline(100.e-6,  ymax=0.7,  c='g', ls='--')

    # Add text annotations above each line
    ax.text(1.e-9, 5e17, 'Bunch length', color='r', ha='center', va='bottom', rotation=90)
    ax.text(25.e-9, 5e17, 'Bunch spacing', color='darkorange', ha='center', va='bottom', rotation=90)
    ax.text(100.e-6, 5e17, 'LHC turn', color='g', ha='center', va='bottom', rotation=90)

    ax.set_xlim(1e-15, 1e-1)
    ax.set_ylim(1e12, 1e20)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_title('LHC wake model at injection')

    plt.tight_layout()
    plt.savefig('001_wake_for_tracking.png')

### Generate the one turn map
* The tracking is performed turn-by-turn by collapsing all the LHC elements in one location: one-turn map
* This means, we don't track through the full lattice
* However, it is a good approximation to simulate collective effects due to wakefields
* Once the one turn map is defined, we can compute the twiss to optain the synchrotron tune $Q_s$ and slip factor $\eta$

In [ ]:
# Create the reference particle for xtrack
reference_particle = xt.Particles(mass0=xp.PROTON_MASS_EV, p0c=p0c)
beta0 = reference_particle.beta0[0]
gamma0 = reference_particle.gamma0[0]

# The line for Q' = 0 will be used for the twiss
segment_map = xt.LineSegmentMap(
    length=circumference,
    betx=machine_radius/Qx, bety=machine_radius/Qy,
    dnqx=[Qx_frac, 0], dnqy=[Qy_frac, 0],
    longitudinal_mode='linear_fixed_rf',
    voltage_rf=V_RF, frequency_rf=f_RF,
    lag_rf=dphi_RF, momentum_compaction_factor=alphap
    )

# Construct the full OTM with line segments and WF elements
one_turn_map_elements = [segment_map,]

# Compile the line
reference_line = xt.Line(one_turn_map_elements)
reference_line.particle_ref = reference_particle

# Compute the twiss parameters
tw = reference_line.twiss()
Qs = tw.qs
eta = tw.slip_factor
print(f'Qs = {Qs}, eta = {eta}')

### Perform the tracking over the selected bunch intensities
* For each intensity bin, the LHC segment map, collective monitor and bunch of particle is generated
* The one-turn-map is composed by the LHC segment, the monitor and the wake. 
  * More features could be added e.g. transverse feedback (damper)
* Finally, the matched bunch of particles is tracked over the defined `number_of_turns`
  * The turn-by-turn data is saved in an HDF5 file inside 📁`bunchmonitor_data_hllhc/`

<div style="text-align:center">
  <img src="data/one-turn-map.png" width="400">
</div>

* $\rightarrow$ [See X. Buffat's II CAS lecture](https://indico.cern.ch/event/1466612/contributions/6448850/)

In [ ]:
%%capture --no-display --no-stdout

results_folder=f'bunchmonitor_data_hllhc'

for bunch_intensity in bunch_intensity_scan:
    print(f'Simulation for b_int = {bunch_intensity:.2e}')

    if (not os.path.isfile(f'./{results_folder}/bunchmonitor_bint_{bunch_intensity:.2e}_bunches.h5')) or flag_restart_sim:
        print(f'File {results_folder}/bunchmonitor_bint_{bunch_intensity:.2e}_bunches.h5 not found or flag_restart_sim active, starting simulation')
        os.makedirs(f'./{results_folder}', exist_ok=True)

        segment_map = xt.LineSegmentMap(
            length=circumference,
            betx=machine_radius/Qx, bety=machine_radius/Qy,
            dnqx=[Qx_frac, chromaticity], dnqy=[Qy_frac, chromaticity],
            longitudinal_mode='linear_fixed_rf',
            voltage_rf=V_RF, frequency_rf=f_RF,
            lag_rf=dphi_RF, momentum_compaction_factor=alphap
            )
        # Create monitors at each RF station
        # initialize a monitor for the average transverse positions
        flush_data_every = int(500)
        particle_monitor_mask = np.full(n_macroparticles, False, dtype=bool)
        particle_monitor_mask[0:5] = True
         
        monitor = xw.CollectiveMonitor(
            base_file_name=f'./{results_folder}/bunchmonitor_with_bellows_bint_{bunch_intensity:.2e}',
            monitor_bunches=True,
            monitor_slices=True,
            monitor_particles=False,
            particle_monitor_mask=particle_monitor_mask,
            flush_data_every=flush_data_every,
            stats_to_store=['mean_x', 'mean_y', 'mean_px', 'sigma_x', 'epsn_x', 'num_particles'],
            stats_to_store_particles=['x', 'px'],
            backend='hdf5',
            zeta_range=(-0.3*bucket_length, 0.3*bucket_length),
            num_slices=num_slices//2,
            bunch_spacing_zeta=circumference,
        )

        # Construct the full OTM with line segments and WF elements
        one_turn_map_elements = [monitor, segment_map, wake_for_tracking]

        # Compile the line
        line = xt.Line(one_turn_map_elements)
        line.particle_ref = reference_particle
        line.build_tracker()

        # initialize a matched gaussian bunch
        particles = xp.generate_matched_gaussian_bunch(
            num_particles=n_macroparticles,
            total_intensity_particles=bunch_intensity,
            nemitt_x=nemitt_x, nemitt_y=nemitt_y,
            sigma_z=sigma_z,
            line=line,
        )

        # apply a kick to the particles
        particles.x += initial_offset

        turn_range = np.arange(0, number_of_turns, 1)

        # Track
        line.track(particles, num_turns=number_of_turns, with_progress=True)
    
    else:
        print(f'File bunchmonitor_with_bellows_bint_{bunch_intensity:.2e}.h5 found, skipping simulation')

### Plotting the transverse turn-by-turn position
* By reading the generated HDF5 file and extracting the average horizontal position $\overline{x}$
  * ❓Can you estimate the intensity threshold of the TMCI instability?
  * ❓How could we plot the bunch intensity over the turns? When are the losses triggered?

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

cmap = plt.get_cmap('jet')
colors = cmap(np.linspace(0.9, 0.1, len(bunch_intensity_scan)))

for i, bint in enumerate(reversed(bunch_intensity_scan)):
    file_path = f'./{results_folder}/bunchmonitor_with_bellows_bint_{bint:.2e}_bunches.h5'
    try:
        with h5py.File(file_path, 'r') as h5file:
            mean_x = h5file['0']['mean_x'][:]
            ax.plot(np.real(mean_x), color=colors[i], label=f'{bint:.2e}', alpha=0.8)
    except FileNotFoundError:
        print(f'File not found: {file_path}')

ax.set_ylim(-1e-3, 1e-3)
ax.set_yscale('linear')
ax.set_xlabel('Turn')
ax.set_ylabel('Mean x position [m]')
ax.set_title('Mean x position vs Turn for all bunch intensities')
ax.legend(title='Bunch Intensity', fontsize='small', loc='best')
fig.tight_layout()
plt.savefig('001_turn_by_turn_data_with_bellows_all_bints.png')
plt.show()

In [ ]:
# [OPTIONAL] Create your intensity plot here! Hint: 'num_particles'

### Plotting the tune spectra
* By performing an FFT of the (windowed) transverse average position for each intensity 
* Over the betatron fractional tune $Q_x$ (red line), and the +/- syncrotron tune $Q_s$ lines
  * ❓What do you observe in the tune spectra when cromaticity $Q'$ is increased?

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

# Show the synchrotron sideband expected position around the tune for all the intensities
ax.vlines([Qx_frac + ii*Qs for ii in range(-5, 6)], ymin=-0.1, ymax=1.1, color='xkcd:grey',
          linestyle='--', label='Synchrotron lines')
ax.vlines([Qx_frac], ymin=-0.1, ymax=1.1, linestyle='--', color='xkcd:red orange')

n_turns_fft = number_of_turns
window = np.kaiser(n_turns_fft, beta=14)

cmap = plt.get_cmap('jet')
colors = cmap(np.linspace(0.1, 0.9, len(bunch_intensity_scan)))

for i, bint in enumerate(bunch_intensity_scan):
    file_path = f'./{results_folder}/bunchmonitor_with_bellows_bint_{bint:.2e}_bunches.h5'
    try:
        with h5py.File(file_path, 'r') as h5file:
            mean_x = h5file['0']['mean_x'][:]
            if len(mean_x) >= n_turns_fft:
                x_values = np.fft.rfftfreq(n_turns_fft, d=1.0)
                y_values = np.abs(np.fft.rfft(mean_x[-n_turns_fft:] * window))
                y_norm = y_values / np.max(y_values)
                ax.plot(x_values, y_norm, color=colors[i], label=f'{bint:.2e}', alpha=0.8)
                ax.scatter(x_values[np.argmax(y_values)], 1, color=colors[i], zorder=10)
            else:
                print(f'Not enough turns for FFT in file: {file_path}')
    except FileNotFoundError:
        print(f'File not found: {file_path}')

ax.set_xlabel('$Q_x$ [-]')
ax.set_ylabel('Amplitude [-]')
ax.set_xlim(0.25, 0.3)
ax.set_ylim(-0.1, 1.1)
ax.legend(title='Bunch Intensity', fontsize='small', loc='best')
fig.suptitle('FFT of the x position for all bunch intensities')
fig.tight_layout()
plt.savefig('001_fft_x_position_all_bints.png')
plt.show()


### Plotting the tune modes in 2D
* Compute the modes from the tune spectra for each intesity: compare the tune-shift with the synchrotron tune
  * ❓At which intensity the mode 0 couples with the mode -1? Does it coincide with the observations in the average transverse position growth?

In [ ]:
# Show the tune-slope and mode coupling in a 2D plot
fig, ax = plt.subplots(figsize=(8, 6))

for bint in bunch_intensity_scan:
    with h5py.File(f'./{results_folder}/bunchmonitor_with_bellows_bint_{bint:.2e}_bunches.h5', 'r') as h5file:
        mean_x = h5file['0']['mean_x'][:]

    n_turns_fft = number_of_turns
    window = np.kaiser(n_turns_fft, beta=8)

    x_values = np.linspace(bint-delta_bint/2, bint+delta_bint/2, 2)/1e11
    
    y_values = (np.fft.rfftfreq(len(mean_x[-n_turns_fft:]), d=1.0) - Qx_frac) / Qs
    
    z_values = np.abs(np.fft.rfft(mean_x[-n_turns_fft:]*window))
    z_values = (z_values / np.max(z_values))

    X, Y = np.meshgrid(x_values, y_values)
    Z = np.tile(z_values, (2, 1)).T

    ax.contourf(X, Y, Z, cmap='YlOrRd')

ax.set_ylim(-3, 1)

ax.set_xlabel('Bunch intensity [$10^{11} p^{+}$]')
ax.set_ylabel('Tune shift $\Delta Q/Q_s$')

fig.suptitle('Tune shift versus intensity,\n Xwakes with resistive wall + bellows')

fig.tight_layout()

### Plotting the intrabunch motion
* Xwakes' `CollectiveMonitor` also allows to save the bunch slices data i.e., the position of every transverse slice.
<div style="text-align:center">
  <img src="data/bunch-slices.png" width="400">
</div>

  * $\rightarrow$ [See X. Buffat's II CAS lecture](https://indico.cern.ch/event/1466612/contributions/6448850/)

* By plotting the transverse position amplitude vs the longitudinal position in the bunch (or slice number), one can reconstruct the **intrabunch motion**
  * ❓ Do you observe any similarity with the Sacherer Headtail modes?
  * ❓ What changes in the intrabunch motion when chromaticity is increased?

<div style="text-align:center">
  <img src="data/sacherer-modes.png" width="400">
</div>

  * $\rightarrow$ [See X. Buffat's I CAS lecture](https://indico.cern.ch/event/1466612/contributions/6448846/)

In [ ]:
for bint in bunch_intensity_scan:

    # Open the file with the slice data
    with h5py.File(f'./{results_folder}/bunchmonitor_with_bellows_bint_{bint:.2e}_slices.h5', 'r') as h5file:
        mean_x = h5file['0']['mean_x'][:]
        num_particles = h5file['0']['num_particles'][:]

    turn_to_start_plot = 800
    num_turn_to_plot = 100

    turn_range_to_plot = np.arange(turn_to_start_plot, turn_to_start_plot + num_turn_to_plot, 1)

    color_range = np.linspace(0.3, 1, num_turn_to_plot)

    fig, ax = plt.subplots(figsize=(6, 4))

    for ii_turn in turn_range_to_plot:

        ax.plot((mean_x * num_particles)[ii_turn, :], color=plt.cm.Blues(color_range[ii_turn-turn_to_start_plot]))

    ax.set_xlabel('Slice number [-]')
    ax.set_ylabel('Headtail signal [-]')

    fig.suptitle(f'Headtail signal for bint = {bint:.2e}')
